In [ ]:
import $ivy.`com.softwaremill.sttp::core:1.3.0-RC5`
import $ivy.`org.json4s::json4s-native:3.6.0`

In [ ]:
import com.softwaremill.sttp._
implicit val backend = HttpURLConnectionBackend()

In [ ]:
import org.json4s._
import org.json4s.native.JsonMethods._
implicit lazy val formats = DefaultFormats

In [ ]:
case class Token(token: String)
implicit val token = Token("c0feebabe") // no, this is not an actual, valid token.

In [ ]:
case class Identifiable(id: Int, name: String)

In [ ]:
def q(s: Any): String = '"' + s.toString + '"'

In [ ]:
def doapiKeys(implicit ev: Token): Seq[BigInt] = {
  val request  = sttp.auth.bearer(ev.token).get(uri"https://api.digitalocean.com/v2/account/keys")
  val response = request.send()
  val json = parse(response.body.right.get)
  for { JObject(ids) <- (json \\ "id"); JField("id", JInt(id)) <- ids } yield id
}

In [ ]:
def doapiDomainRecords(domainname: String)(implicit ev: Token): JValue = {
  val request  =
    sttp.auth.bearer(ev.token)
      .get(uri"https://api.digitalocean.com/v2/domains/${domainname}/records")
  val response = request.send()
  val json = parse(response.body.right.get)
  json
}

In [ ]:
def doapiDomainRecordDelete(domainname: String, hostnames: String*)(implicit ev: Token): Seq[Boolean] = {
  val records = (doapiDomainRecords(domainname)(ev) \\ "domain_records").extract[List[Identifiable]]
  hostnames
    .flatMap { hostname =>
      records
        .filter(r => r.name == hostname)
        .map { r =>
          // println(s"https://api.digitalocean.com/v2/domains/${domainname}/records/${r.id}")
          val request  =
             sttp.auth.bearer(ev.token)
              .delete(uri"https://api.digitalocean.com/v2/domains/${domainname}/records/${r.id}")
          val response = request.send()
          response.isSuccess
        }
    }
}

In [ ]:
function digitalocean_domain_record_create {
  local hostname=$1
  local domainname=$2
  local fqdn=${hostname}.${domainname}

  http GET https://api.digitalocean.com/v2/droplets Authorization:"Bearer ${doapi_token}" --ignore-stdin | \
    jq -r ".droplets[] | select(.name == \"${hostname}\") | .networks.v4[0].ip_address" | head -1 | \
      while read ip ;do
        digitalocean_json_domain_create A ${hostname} ${ip} 300 > ${logdir}/domain_records_create_A-${hostname}-request.json
        cat ${logdir}/domain_records_create_A-${hostname}-request.json | \
          http POST https://api.digitalocean.com/v2/domains/${domainname}/records Authorization:"Bearer ${doapi_token}" > ${logdir}/domain_records_create_A-${hostname}-response.json
      done

  http GET https://api.digitalocean.com/v2/droplets Authorization:"Bearer ${doapi_token}" --ignore-stdin | \
    jq -r ".droplets[] | select(.name == \"${hostname}\") | .networks.v6[0].ip_address" | head -1 | \
      while read ip ;do
        digitalocean_json_domain_create AAAA ${hostname} ${ip} 300 > ${logdir}/domain_records_create_AAAA-${hostname}-request.json
        cat ${logdir}/domain_records_create_AAAA-${hostname}-request.json | \
          http POST https://api.digitalocean.com/v2/domains/${domainname}/records Authorization:"Bearer ${doapi_token}" > ${logdir}/domain_records_create_AAAA-${hostname}-response.json
      done
}

In [ ]:
def doapiDomainRecordCreate(domainname: String, hostname: String)(implicit ev: Token): Seq[Boolean] = ???

In [ ]:
def doDropletCreateJson(domainname: String, hostname: String,
                        region: String, size: String, image: Int, ipv6: Boolean, tags: String)
                       (implicit ev: Token): String = {
  val keys = doapiKeys(ev).map(s => q(s)).mkString(", ")
  val _tags = tags.split(",").map(s => q(s)).mkString(", ")
  val json =
s"""{
   ^  "name": "${hostname}",
   ^  "region": "${region}",
   ^  "size": "${size}",
   ^  "image": ${image},
   ^  "ssh_keys": [ ${keys} ],
   ^  "backups": false,
   ^  "ipv6": ${ipv6},
   ^  "private_networking": null,
   ^  "volumes": null,
   ^  "tags": [ ${_tags} ],
   ^  "user_data": "#cloud-config
   ^manage_etc_hosts: false
   ^write_files:
   ^  - path: /etc/resolvconf/resolv.conf.d/head
   ^    content: |
   ^      domain ${domainname}
   ^runcmd:
   ^  - service resolvconf restart
   ^"
   ^}""".stripMargin('^')
  //println(json)                         
  val check = parse(json)
  //println(check)
  json    
}

In [ ]:
def doDomainRecordsCreateJson(type: String, fqdn: String, address: String, ttl: Int)
                             (implicit ev: Token): String = {
  val json =
s"""{
   ^  "type": "${type}",
   ^  "name": "${fqdn}",
   ^  "data": "${address}",
   ^  "ttl": ${ttl}
   ^}""".stripMargin('^')
  //println(json)                         
  val check = parse(json)
  //println(check)
  json    
}

In [ ]:
function digitalocean_json_domain_create {
local type=$1
local name=$2
local address=$3
local ttl=$4
cat << EOD
{
  "type": "${type}",
  "name": "${name}",
  "data": "${address}",
  "ttl": ${ttl}
}
EOD
}


In [ ]:
def doDropletCreate(domainname: String, hostname: String, region: String,
                    size: String, image: Int, ipv6: Boolean, tags: String)
                   (implicit ev: Token) = {
  val json = doDropletCreateJson(domainname, hostname, region, size, image, ipv6, tags)(ev)
  val request  =
    sttp.auth.bearer(ev.token)
      .post(uri"https://api.digitalocean.com/v2/droplets")
      .contentType("application/json")
      .body(json)
  val response = request.send()
  response
}

In [ ]:
def doapiDroplets(implicit ev: Token): JValue = {
  val request  =
    sttp.auth.bearer(ev.token)
      .get(uri"https://api.digitalocean.com/v2/droplets")
  val response = request.send()
  // println(response)
  val json = parse(response.body.right.get)
  //println(json)
  json
}

In [ ]:
def doapiDropletDelete(domainname: String, hostnames: String*)(implicit ev: Token): Seq[Boolean] = {
  val records = (doapiDroplets(ev) \\ "droplets").extract[List[Identifiable]]
  hostnames
    .flatMap { hostname =>
      records
        .filter(r => r.name == hostname)
        .map { r =>
          //println(s"https://api.digitalocean.com/v2/droplets/${r.id}")
          val request  =
             sttp.auth.bearer(ev.token)
              .delete(uri"https://api.digitalocean.com/v2/droplets/${r.id}")
          val response = request.send()
          response.isSuccess
        }
    }
}

In [ ]:
doDropletCreate("example.com", "test", "LON1", "s-1vcpu-1gb", 36951366, true, "dev,rgomes")

In [ ]:
doapiDropletDelete("example.com", "test")